# MOUNTING COLAB

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# INSTALLING LIBRARIES

In [ ]:
!pip3 install easyocr

# IMPORTING LIBRARIES

In [ ]:
import torch
import cv2
import time
import re
import numpy as np
import easyocr

# DEFINING GLOBAL VARIABLES

In [ ]:
EASY_OCR = easyocr.Reader(['en']) ### initiating easyocr
OCR_TH = 0.2

# FUNCTIONS

## Function to read input details from user

In [ ]:
def input_details():

  # Read license plate number from user
  vehicle_license_number  = input("Enter License Plate Number : ")

  # # Number of review pages to be checked
  vid_path = input("Enter path of the video file : ")

  # Read number of frames per second needed
  frame_frequency = int(input("Enter number of frames per second needed : "))

  return vehicle_license_number, vid_path, frame_frequency

## Function for running detection

In [ ]:
def detectx (frame, model):
    frame = [frame]
    results = model(frame)

    labels, cordinates = results.xyxyn[0][:, -1], results.xyxyn[0][:, :-1]

    return labels, cordinates

## Function to plot Bounting Box and get coordinates

In [ ]:
def plot_boxes(results, frame, classes):

    labels, cord = results
    n = len(labels)
    x_shape, y_shape = frame.shape[1], frame.shape[0]


    ### looping through the detections
    for i in range(n):
        row = cord[i]
        if row[4] >= 0.55:
            x1, y1, x2, y2 = int(row[0]*x_shape), int(row[1]*y_shape), int(row[2]*x_shape), int(row[3]*y_shape) ## BBOx coordniates
            text_d = classes[int(labels[i])]
            # cv2.imwrite("./output/dp.jpg",frame[int(y1):int(y2), int(x1):int(x2)])

            coords = [x1,y1,x2,y2]
            
            recognize_plate_easyocr(img = frame, coords= coords, reader= EASY_OCR, region_threshold= OCR_TH)




## Function to recognize the license plates

In [ ]:
def recognize_plate_easyocr(img, coords,reader,region_threshold):
    # separate coordinates from box
    xmin, ymin, xmax, ymax = coords
    nplate = img[int(ymin):int(ymax), int(xmin):int(xmax)] ### cropping the number plate from the whole image


    ocr_result = reader.readtext(nplate)


    filter_text(region=nplate, ocr_result=ocr_result, region_threshold= region_threshold)


## Function to get timecode from frame number

In [ ]:
def frame_timecode(frame_number):
  # frame_hours = frame_number //(fps * 3600)
  frame_minutes = frame_number // (fps * 60) % 60
  frme_seconds =  frame_number // fps % 60

  frame_time_std = frame_minutes + frme_seconds/100

  return frame_time_std

## Function to check for the desired number plate

In [ ]:
def check_plate_num(plate_number):

  if vehicle_license_number in plate_number:
      frame_time = frame_timecode(frame_no)
      final_time = timestamp[-1][1]
      if final_time == 0:
        timestamp[-1] = [frame_time, frame_time]
      elif frame_time - final_time > 0.1:
        timestamp.append([frame_time, frame_time])
      else:
        timestamp[-1][1] = frame_time
      print("Timestamps : ", timestamp)

## Function to filter out wrong detections

In [ ]:
def filter_text(region, ocr_result, region_threshold):

    rectangle_size = region.shape[0]*region.shape[1]
    for result in ocr_result:
        length = np.sum(np.subtract(result[0][1], result[0][0]))
        height = np.sum(np.subtract(result[0][2], result[0][1]))
        
        if length*height / rectangle_size > region_threshold:
            # plate.append(result[1])
            check_plate_num(result[1])


# Main

In [ ]:
vehicle_license_number, vid_path, frame_frequency = input_details()
timestamp = [[0,0]]

print("[INFO] Loading the model... ")

## loading the custom trained model
# Kindly replace '/content/drive/MyDrive/yolov5-master' with location of your own yolov5-master.
# Download the last.pt present in this directory and replace '/content/drive/MyDrive/deep_learning/personal_projects/number_plate_yolo/last.pt' with your path to the uploaded last.pt.
model =  torch.hub.load('/content/drive/MyDrive/yolov5-master', 'custom', source ='local', path='/content/drive/MyDrive/deep_learning/personal_projects/number_plate_yolo/last.pt',force_reload=True)
print("Model loaded successfully...")

classes = model.names ### class names in string format


if vid_path != None:
    print(f"[INFO] Working with video: {vid_path}")

    ## reading the video
    cap = cv2.VideoCapture(vid_path)


    # by default VideoCapture returns float instead of int
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_frequency = fps//frame_frequency

    frame_no = 1

    while True:
        ret, frame = cap.read()
        if ret:
          if frame_no % frame_frequency == 0:
            # print(f"[INFO] Working with frame {frame_no} ")

            frame = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
            results = detectx(frame, model = model)
            frame = cv2.cvtColor(frame,cv2.COLOR_RGB2BGR)

            plot_boxes(results, frame,classes = classes)
          frame_no += 1
        else:
          break
    if timestamp[0][1] == 0:
      print('The vehicle could not be spotted in the given video.')
    else:
      print("The vehicle appeared in the video at timestamps : \n\n")
      for time_instance in timestamp:
        print(f"\t From {time_instance[0]} to {time_instance[1]}")
